In [15]:
import numpy as np

"""
    0 con1 1 con3 2 max3 3 input 4 output 5 None
"""
op_name = ['con1', 'con3', 'max3', 'input', 'output', 'None']
"""
    Maximum Turbo Boost Frequency, Number of Cores, Number of Threads, Level 3 Cache, Maximum Memory Bandwidth
"""
device_fetures = ['CPU Clock Frequency', 'Maximum Turbo Boost Frequency', 'Number of Cores', 'Number of Threads', 'Level 3 Cache', ', Maximum Memory Bandwidth']

In [36]:
import torch
from utils.plotter import MetricsPlotter
from utils.logger import Logger
from utils.utils import set_settings
from data import experiment
from utils.config import get_config

args = get_config()
set_settings(args)
args.train_size = 200
# logger plotter
exper_detail = f"Dataset : {args.dataset.upper()}, Model : {args.model}, Train_size : {args.train_size}, Bs : {args.bs}, Rank : {args.rank}, "
exper_detail += f"Train Device : {args.train_device} "
log_filename = f'Model_{args.model}_T{args.train_size}_R{args.rank}'
log = Logger(log_filename, exper_detail, args)
plotter = MetricsPlotter(log_filename, args)
args.log = log
log(str(args.__dict__))

|2024-10-03 18:15:44| {
     'att': 0, 'bs': 64, 'classification': False, 'cross': 0,
     'dataset': cpu, 'debug': False, 'decay': 0.0001, 'density': 0.8,
     'device': cpu, 'device_name': core-i7-7820x, 'epochs': 1000, 'eval_device': desktop-cpu-core-i9-13900k-fp32,
     'experiment': False, 'graph_encoder': gat, 'heads': 8, 'inductive': False,
     'llm': 1, 'logger': None, 'loss_func': L1Loss, 'lr': 0.001,
     'model': ours, 'op_encoder': embed, 'optim': AdamW, 'order': 4,
     'path': ./datasets/, 'patience': 100, 'program_test': False, 'rank': 100,
     'record': True, 'retrain': False, 'rounds': 5, 'seed': 0,
     'train_device': desktop-cpu-core-i7-7820x-fp32, 'train_size': 200, 'verbose': 0, 'visualize': True,
}
|2024-10-03 18:15:44| {'classification': False, 'visualize': True, 'inductive': False, 'bs': 64, 'lr': 0.001, 'decay': 0.0001, 'loss_func': 'L1Loss', 'optim': 'AdamW', 'path': './datasets/', 'dataset': 'cpu', 'train_size': 200, 'density': 0.8, 'logger': 'None', 'mode

In [37]:
from train_model import Model
from data import DataModule

# Initialize
exper = experiment(args)
datamodule = DataModule(exper, args)
model = Model(datamodule, args)

desktop-cpu-core-i7-7820x-fp32.pickle
|2024-10-03 18:15:44| Train_length : 200 Valid_length : 1528 Test_length : 13556 Max_value : 0.01
加载存储数据 tensor([[0.6143, 0.2500, 0.5000, 0.2750, 0.3750]])


In [47]:
model.load_state_dict(torch.load(f'checkpoints/ours/Model_ours_cpu_S{args.train_size}_R100_round_2.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [48]:
from tqdm import *
idx = 96
sample_input = datamodule.train_loader.dataset[idx]
graph, features, values = sample_input
features = features.reshape(1, -1)
print(graph, features)
from time import time 
t1 = time()
nums = 1000
for i in trange(nums):
    model(graph, features)
t2 = time()
print((t2 - t1) / nums)

Graph(num_nodes=9, num_edges=39,
      ndata_schemes={}
      edata_schemes={}) tensor([[5, 3, 1, 2, 2, 0, 0, 0, 4]])


100%|██████████| 1000/1000 [00:02<00:00, 357.87it/s]

0.0027961938381195067


In [49]:
model.model.llms.aug_data

tensor([[0.6143, 0.2500, 0.5000, 0.2750, 0.3750]], requires_grad=True)

In [50]:
from tqdm import trange


def get_embeds_importance(model, sample_input):
    import torch
    
    graph, features, values = sample_input
    features = features.reshape(1, -1).long()
    
    # 清除模型中所有参数的梯度
    model.zero_grad()
    
    # 确保 info_embeds 和 graph_embeds 的梯度可以计算
    batch_size = features.shape[0]
    info_embeds = model.model.llms(batch_size)
    graph_embeds = model.model.graph_encoder(graph, features)
    
    # 确保在前向传播之前设置 requires_grad=True
    info_embeds.requires_grad_(True)
    graph_embeds.requires_grad_(True)
    
    # 在非叶子张量上保留梯度
    info_embeds.retain_grad()
    graph_embeds.retain_grad()
    
    # 组合嵌入，进行前向传播
    embeds = torch.cat([graph_embeds, info_embeds], dim=1)
    output = model.model.predictor(embeds)
    
    # 使用人为定义的损失函数，比如对输出的简单加权平均
    loss = output.mean()  # 简单的均值作为损失
    
    # 反向传播计算损失的梯度
    loss.backward()
    
    # 获取两个嵌入的梯度并计算它们的重要性
    info_importance = info_embeds.grad.abs().sum().item()
    graph_importance = graph_embeds.grad.abs().sum().item()
    
    total_importance = info_importance + graph_importance
    graph_importance /= total_importance
    info_importance /= total_importance
    
    # print("LLMs Embeds Importance:", info_importance)
    # print("Graph Embeds Importance:", graph_importance)
    return graph_importance, info_importance

# 全局变量
llms_total, gread_total = 0, 0
total_samples = 0

# 处理训练集
for i in trange(len(datamodule.train_loader.dataset)):
    g, l = get_embeds_importance(model, sample_input=datamodule.train_loader.dataset[i])
    gread_total += g
    llms_total += l
total_samples += len(datamodule.train_loader.dataset)
llms_avg_train = llms_total / total_samples
gread_avg_train = gread_total / total_samples
print("LLMs Embeds Importance (Train):", llms_avg_train)
print("Graph Embeds Importance (Train):", gread_avg_train)

# # 处理验证集
# for i in trange(len(datamodule.valid_loader.dataset)):
#     g, l = get_embeds_importance(model, sample_input=datamodule.valid_loader.dataset[i])
#     gread_total += g
#     llms_total += l
# total_samples += len(datamodule.valid_loader.dataset)
# llms_avg_valid = llms_total / total_samples
# gread_avg_valid = gread_total / total_samples
# print("LLMs Embeds Importance (Train + Valid):", llms_avg_valid)
# print("Graph Embeds Importance (Train + Valid):", gread_avg_valid)
#
# # 处理测试集
# for i in trange(len(datamodule.test_loader.dataset)):
#     g, l = get_embeds_importance(model, sample_input=datamodule.test_loader.dataset[i])
#     gread_total += g
#     llms_total += l
# total_samples += len(datamodule.test_loader.dataset)
# llms_avg_test = llms_total / total_samples
# gread_avg_test = gread_total / total_samples
# print("LLMs Embeds Importance (All):", llms_avg_test)
# print("Graph Embeds Importance (All):", gread_avg_test)

100%|██████████| 200/200 [00:01<00:00, 134.42it/s]

LLMs Embeds Importance (Train): 0.38952354947415574
Graph Embeds Importance (Train): 0.6104764505258443


In [51]:
def get_device_feature_importance(model, sample_input):
    import torch
    
    # 清除模型中所有参数的梯度
    model.zero_grad()
    
    graph, features, values = sample_input
    
    # 获取 info_importance
    _, info_importance = get_embeds_importance(model, sample_input)
    
    # 设置 aug_data 为 requires_grad=True
    model.model.llms.aug_data.requires_grad = True
    
    # 前向传播计算输出 y
    features = features.reshape(1, -1).long()
    output = model(graph, features)
    
    # 反向传播计算每个输入特征的梯度
    output.backward()
    
    # 获取 aug_data 的梯度并计算重要性
    aug_data_importance = model.model.llms.aug_data.grad.abs()
    
    # 标准化重要性，使其和为1
    aug_data_importance = aug_data_importance / aug_data_importance.sum()
    
    # 打印 aug_data 中每个元素的重要性
    each_device_info_importance = aug_data_importance.squeeze(0).detach().numpy()
    print("Overall importance of each element in aug_data to the model:")
    for i in range(len(each_device_info_importance)):
        print(f"LLMs features-{device_fetures[i]} : {each_device_info_importance[i]:.4f}", )
    # 计算每个 aug_data 元素在整个模型中的重要性
    # each_device_info_importance = info_importance * aug_data_importance
    # each_device_info_importance = each_device_info_importance.squeeze(0).detach().numpy()
    # print("Overall importance of each element in aug_data to the model:")
    # for i in range(len(each_device_info_importance)):
    #     print(f"LLMs features-{device_fetures[i]} : {each_device_info_importance[i]:.4f}", )
    # print('-' * 80)

    return each_device_info_importance

all_device_importance = np.zeros(5)
for i in range(len(datamodule.train_loader.dataset)):
    temp = get_device_feature_importance(model = model, sample_input = datamodule.train_loader.dataset[i])
    all_device_importance += temp 
all_device_importance /= len(datamodule.train_loader.dataset)

Overall importance of each element in aug_data to the model:
LLMs features-CPU Clock Frequency : 0.5336
LLMs features-Maximum Turbo Boost Frequency : 0.0102
LLMs features-Number of Cores : 0.2489
LLMs features-Number of Threads : 0.0555
LLMs features-Level 3 Cache : 0.1519
Overall importance of each element in aug_data to the model:
LLMs features-CPU Clock Frequency : 0.5329
LLMs features-Maximum Turbo Boost Frequency : 0.0096
LLMs features-Number of Cores : 0.2498
LLMs features-Number of Threads : 0.0545
LLMs features-Level 3 Cache : 0.1532
Overall importance of each element in aug_data to the model:
LLMs features-CPU Clock Frequency : 0.5316
LLMs features-Maximum Turbo Boost Frequency : 0.0101
LLMs features-Number of Cores : 0.2496
LLMs features-Number of Threads : 0.0538
LLMs features-Level 3 Cache : 0.1549
Overall importance of each element in aug_data to the model:
LLMs features-CPU Clock Frequency : 0.5302
LLMs features-Maximum Turbo Boost Frequency : 0.0111
LLMs features-Number 

In [52]:
all_device_importance

array([0.45456609, 0.03747935, 0.22867044, 0.02022622, 0.25905791])

In [53]:
for i in range(len(all_device_importance)):
    print(f"{all_device_importance[i] * 100:.2f}\%", end=' & ')

45.46\% & 3.75\% & 22.87\% & 2.02\% & 25.91\% & 

In [11]:
import random


def get_node_importance(model, sample_input):
    graph, features, values = sample_input
    features = features.reshape(1, -1).long()
    g, feats = graph, model.model.graph_encoder.dnn_embedding(features).reshape(features.shape[0] * 9, -1)
    attention_weights = []
    for i, (layer, norm, act) in enumerate(zip(model.model.graph_encoder.layers, model.model.graph_encoder.norms, model.model.graph_encoder.acts)):
        # 获取 GAT 的注意力权重
        feats, attn = layer(g, feats, get_attention=True)
        feats = feats.mean(dim=1)  # 聚合多个头的输出
        feats = norm(feats)
        feats = act(feats)
        attention_weights.append(attn)
    batch_sizes = torch.as_tensor(g.batch_num_nodes()).to(model.model.args.device)  # 每个图的节点数
    first_nodes_idx = torch.cumsum(torch.cat((torch.tensor([0]).to(model.model.args.device), batch_sizes[:-1])), dim=0)  # 使用torch.cat来连接Tensor
    first_node_features = feats[first_nodes_idx]
    return first_node_features, attention_weights


def aggregate_node_attention(model, sample_input, graph, global_node_attention):
    # 获取图中的节点数和边数
    num_nodes = graph.number_of_nodes()
    num_edges = graph.number_of_edges()

    # 假设 global_node_attention 的形状是 [num_edges, num_heads, 1]
    mean_attention = global_node_attention.mean(dim=1).squeeze(-1)  # [num_edges]
    
    # print("Mean attention per edge:", mean_attention)
    
    # 初始化每个节点的注意力权重为零
    node_importance = torch.zeros(num_nodes, device=mean_attention.device)
    
    # 遍历每条边，将注意力权重聚合到对应的节点上
    for edge_idx in range(num_edges):
        src, dst = graph.find_edges(edge_idx)  # 获取边的起始节点和目标节点
        if dst == 0:
            # 累加目标节点的注意力权重
            node_importance[src] += mean_attention[edge_idx]
            # print(f"Edge {edge_idx:2d}: src={src}, dst={dst}, Attention={mean_attention[edge_idx]:.4f}, accumulated importance={node_importance[src]}")
    
    # 最后可以对 node_importance 进行归一化处理，使其值在合理范围内
    node_importance = node_importance / node_importance.sum()
    
    print("Node importance to the global node:\n", node_importance.detach().cpu().numpy())

    # 获取 graph_importance
    graph_importance, _ = get_embeds_importance(model, sample_input)
    each_device_node_importance = graph_importance * node_importance
    each_device_node_importance = each_device_node_importance.detach().numpy()
    _, key, values = sample_input
    print(key)
    print("Overall importance of each element in graph to the Global Node:")
    for i in range(len(key)):
        print(f"Key-{key[i]}-{op_name[key[i]]}: {each_device_node_importance[i]:.4f}")
    return node_importance


# 使用函数计算每个节点对全局节点的重要性
idx = random.randint(0, len(datamodule.train_loader.dataset))
sample_input = datamodule.train_loader.dataset[idx]
graph, features, values = sample_input
first_node_features, attention_weights = get_node_importance(model, sample_input = datamodule.train_loader.dataset[idx])
node_importance = aggregate_node_attention(model, sample_input, graph, attention_weights[-1]).detach().cpu().numpy()

Node importance to the global node:
 [0.2237378  0.20893955 0.         0.18403508 0.         0.19714415
 0.         0.         0.18614344]
tensor([5, 3, 0, 2, 0, 1, 0, 0, 4])
Overall importance of each element in graph to the Global Node:
Key-5-None: 0.1229
Key-3-input: 0.1148
Key-0-con1: 0.0000
Key-2-max3: 0.1011
Key-0-con1: 0.0000
Key-1-con3: 0.1083
Key-0-con1: 0.0000
Key-0-con1: 0.0000
Key-4-output: 0.1022


In [12]:
all_node_importance = np.zeros(9)
idx = random.randint(0, len(datamodule.train_loader.dataset))
for i in range(len(datamodule.train_loader.dataset)):
    sample_input = datamodule.train_loader.dataset[i]
    graph, features, values = sample_input
    first_node_features, attention_weights = get_node_importance(model, sample_input = datamodule.train_loader.dataset[i])
    node_importance = aggregate_node_attention(model, sample_input, graph, attention_weights[-1]).detach().cpu().numpy()
    break

Node importance to the global node:
 [0.13280469 0.14259605 0.23872896 0.         0.         0.12645777
 0.20422941 0.         0.15518314]
tensor([5, 3, 0, 0, 0, 1, 1, 0, 4])
Overall importance of each element in graph to the Global Node:
Key-5-None: 0.0793
Key-3-input: 0.0852
Key-0-con1: 0.1426
Key-0-con1: 0.0000
Key-0-con1: 0.0000
Key-1-con3: 0.0755
Key-1-con3: 0.1220
Key-0-con1: 0.0000
Key-4-output: 0.0927


In [13]:
import numpy as np
import random

# 初始化存储重要性累积和出现次数的字典
all_node_importance = {i: 0 for i in range(9)}
key_counts = {i: 0 for i in range(9)}

idx = random.randint(0, len(datamodule.train_loader.dataset))
for i in range(len(datamodule.train_loader.dataset)):
    sample_input = datamodule.train_loader.dataset[i]
    graph, features, values = sample_input
    first_node_features, attention_weights = get_node_importance(model, sample_input = datamodule.train_loader.dataset[i])
    node_importance = aggregate_node_attention(model, sample_input, graph, attention_weights[-1]).detach().cpu().numpy()
    
    for key in features.numpy():
        if node_importance[key] == 0:
            continue
        all_node_importance[key] += node_importance[key]
        key_counts[key] += 1  # 统计每个 key 出现的次数
    
# 计算每个 key 的平均值
average_importance = {key: all_node_importance[key] / key_counts[key] for key in all_node_importance if key_counts[key] > 0}

# 打印结果
print("Node importance to the global node:\n", node_importance)
print("tensor:", features)
print("Overall average importance of each element in graph to the Global Node:")
for key, avg_value in average_importance.items():
    print(f"Key-{key}-{op_name[key]}: {avg_value:.4f}")
    

Node importance to the global node:
 [0.17458987 0.13331988 0.1754015  0.         0.         0.10634027
 0.20527203 0.         0.20507644]
tensor([5, 3, 0, 0, 0, 1, 1, 0, 4])
Overall importance of each element in graph to the Global Node:
Key-5-None: 0.1040
Key-3-input: 0.0794
Key-0-con1: 0.1045
Key-0-con1: 0.0000
Key-0-con1: 0.0000
Key-1-con3: 0.0634
Key-1-con3: 0.1223
Key-0-con1: 0.0000
Key-4-output: 0.1222
Node importance to the global node:
 [0.1623798  0.14269248 0.17175214 0.         0.         0.18581527
 0.17338851 0.         0.16397187]
tensor([5, 3, 2, 0, 0, 1, 0, 0, 4])
Overall importance of each element in graph to the Global Node:
Key-5-None: 0.0895
Key-3-input: 0.0786
Key-2-max3: 0.0946
Key-0-con1: 0.0000
Key-0-con1: 0.0000
Key-1-con3: 0.1024
Key-0-con1: 0.0955
Key-0-con1: 0.0000
Key-4-output: 0.0903
Node importance to the global node:
 [0.16303375 0.15230024 0.15737665 0.18693058 0.         0.17008786
 0.         0.         0.17027092]
tensor([5, 3, 1, 1, 0, 1, 0, 0, 4])

In [14]:
for key, avg_value in average_importance.items():
    print(f"{average_importance[key] * 100:.2f}\%", end=' & ')

19.69\% & 14.39\% & 14.13\% & 14.41\% & 15.79\% & 14.12\% & 